In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
torch.manual_seed(42)


In [2]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('../Data/research-abstracts-labeled.csv')

In [4]:
gpt = df[df['label'] == 0]
human = df[df['label'] == 1]

In [5]:
gpt = gpt.sample(n=150)
human = human.sample(n=150)

In [6]:
gpt.head()

,title,label,text,word_count
17884,Learning Novel Policies For Tasks,0,"In this work, we present a reinforcement learn...",168
9568,Nonparametrically consistent depth-based class...,0,We introduce a class of depth-based classifica...,147
10444,Probabilistic Regression of Rotations using Qu...,0,Accurate estimates of rotation are crucial to ...,146
11958,The first spectral line surveys searching for ...,0,Our aim is to observationally investigate the ...,447
19288,Gamma-ray emission states in the redback milli...,0,Long expected transition states between the ro...,226


In [7]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()


In [8]:
import numpy as np
def remove_outliers(embeddings):
        # Calculate the mode of the embeddings
        mode_val = np.mean(embeddings[0], axis=None)
        
        # Replace outliers with the mode value
        embeddings[np.abs(embeddings - mode_val) > 2 * np.std(embeddings)] = mode_val
        
        return embeddings
def min_max_normalize(embeddings):
    # Find the minimum and maximum values in the embeddings
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    
    # Normalize the embeddings to range [0, 255]
    normalized_embeddings = 255 * (embeddings - min_val) / (max_val - min_val)
    
    return normalized_embeddings.astype(np.uint8)  # Convert to uint8 for integer values between 0 and 255

def reshape_embeddings(embeddings):
    # Reshape the embeddings to 3D array
    return embeddings.reshape(32,32)

In [9]:
gpt['embeddings'] = gpt['text'].apply(generate_bert_embeddings)
gpt['normalized_embeddings'] = gpt['embeddings'].apply(min_max_normalize)
gpt['normalized_embeddings1'] = gpt['normalized_embeddings'].apply(remove_outliers)
gpt['reshaped_embeddings'] = gpt['normalized_embeddings1'].apply(reshape_embeddings)

In [10]:
human['embeddings'] = human['text'].apply(generate_bert_embeddings)
human['normalized_embeddings'] = human['embeddings'].apply(min_max_normalize)
human['normalized_embeddings1'] = human['normalized_embeddings'].apply(remove_outliers)
human['reshaped_embeddings'] = human['normalized_embeddings1'].apply(reshape_embeddings)

In [12]:
import matplotlib.pyplot as plt
folder_path = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/output/Research_GPT/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
for i in range(len(gpt['reshaped_embeddings'])):
    plt.imshow(gpt['reshaped_embeddings'].iloc[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig(folder_path+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


In [13]:
import matplotlib.pyplot as plt
folder_path = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/output/Research_Human/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
for i in range(len(human['reshaped_embeddings'])):
    plt.imshow(human['reshaped_embeddings'].iloc[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig(folder_path+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 2, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(64, 32)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.pool3(self.relu3(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x) 

        return x

class CNN_EmbeddingAvg(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 2, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(64, 32)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        outputs = []
        for channel in x.shape[1]:
            x = self.pool1(self.relu1(self.conv1(x)))
            x = self.pool2(self.relu2(self.conv2(x)))
            x = self.flatten(x)
            outputs.append(x)

        stacked_output = torch.stack(outputs, dim = 1)
        embedding_mean = torch.mean(stacked_output, dim = 1)

        y = self.relu4(self.fc1(embedding_mean))
        y = self.relu5(self.fc2(y))
        y = self.fc3(y) 

        return y

In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
import tqdm

In [16]:
data_dir = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/output'

In [17]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

In [18]:
import torch
import torchvision.models as models

# Define the model architecture (make sure it matches the architecture of the saved model)
#model = models.resnet18()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(in_channels=3, num_classes=2).to(device)
# Path to the saved model
model_path = "/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/model/wiki_model.pth"

# Load the saved model parameters into the model
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model.eval()

# Now the model is loaded and ready for inference or further training


CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (relu5): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)

In [ ]:
from PIL import Image
gpt_preds = []
for image in os.listdir(data_dir+"/Research_GPT"):
    img = Image.open(os.path.join(data_dir+"/Research_GPT", image)).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)
    outputs = model(img)
    _, predicted = torch.max(outputs.data, 1)
    print(predicted.item())
    gpt_preds.append(predicted.item())

In [20]:
from collections import Counter
Counter(gpt_preds)

Counter({0: 146, 1: 4})

In [ ]:
from PIL import Image
human_preds = []
for image in os.listdir(data_dir+"/Research_Human"):
    img = Image.open(os.path.join(data_dir+"/Research_Human", image)).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)
    outputs = model(img)
    _, predicted = torch.max(outputs.data, 1)
    print(predicted.item())
    human_preds.append(predicted.item())

In [22]:
from collections import Counter
Counter(human_preds)

Counter({0: 119, 1: 31})